#Приветсвие (ReadMe)

Здравствуйте! Наш алгоритм представлен в виде ноутбука в google collab чуть ниже.


Он включает в себя несколько блоков:

1. Установка зависимосей


2. Загрузка модели : в этом пукте происходит загрузка модели и токинизатора для нее.


3. Работа с файлами : этот блок содержит функции для обработки поступающих резюме в разных форматах (docx, pdf, doc, txt), после предобработки фунции выводят резюме в виде текста. Используется переводчик для  перевода на английский язык(если файл на англ. языке то он возвращает текст на английском языке), но из-за того что в запрос можно поместить ограниченое кол-во символов мы сделали несколько функций, которые разделяют текст до ограниченого кол-ва(3000), далее итарационно подается текст, и сохраняется в буффер(переменная: list_offbuffer), после чего объеденяем через \n.

4. Запись примеров поведения модели : этот блок содержит 5 шаблонов (резюме-хороший ответ), но используем мы только 2 из них в разных конфигурациях. Шаблоны нужны для того, чтобы показать модели в каком формате она должна отвечать.

5. Запуск модели : этот блок содержит загрузку поведенческих шаблонов(правила чтобы модель выводила именно то что нам нужно, загрузка самого резюме загруженного пользователем, которое хотим обработать)

6. Фильтрация вывода модели : Этот блок сосдержит защиту от глюков, запись вывода модели в json

7. Main : это главный блок в основе которого лежит содержатся только главные функции обработки, запуска модели,

8. pipeline : происходит вызов главного блока. Вам сначала предложится выбрать резюме(можно несколько), после этого происходит обработка загруженного файла, и в конце выводится время работы модели. В итоге вы получите сохраненый json в папке output.

# Установка зависимостей

In [1]:
%load_ext autoreload
%autoreload 2
!pip install -U transformers
!pip install accelerate
!pip install bitsandbytes

!apt-get install antiword unrtf poppler-utils tesseract-ocr
!pip install googletrans==3.1.0a0 python-docx pdfminer.six

#accelerate библиотека для эффективного обучения моделей. write_basic_config прописывается, чтобы модель увидела наш TPU и использовала его
!python -c "from accelerate.utils import write_basic_config; write_basic_config()"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  antiword poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd unrtf
0 upgraded, 6 newly installed, 0 to remove and 35 not upgraded.
Need to get 5,166 kB of archives.
After this operation, 17.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 antiword amd64 0.37-16 [118 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:4 http://archive.ubuntu.com/

#Загрузка модели

Импорт библиотек

In [2]:

from googletrans import Translator
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import files
from docx import Document
from pdfminer.high_level import extract_text
from datetime import datetime

import re, unicodedata, json, time, re, torch, transformers, os, subprocess

print(torch.cuda.is_available())
print(transformers.is_bitsandbytes_available())



True
True


Загрузка модели

In [3]:
device = "cuda"
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",quantization_config=bnb_config)#,quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

#Работа с файлами

In [4]:
def upload_file():
    #загрузка файлов
    uploaded_file = files.upload()
    files_name = list(uploaded_file.keys())
    folder_path = os.path.join(os.getcwd(), 'input')
    file_paths = []

    for file_name in files_name:
        source_path = os.path.join(os.getcwd(), file_name)
        dest_path = os.path.join(folder_path, file_name)
        file_paths.append(dest_path)
        os.rename(source_path, dest_path)

    return file_paths, files_name

def cleen_doc(text):
  text = text.replace("|", "")
  text = re.sub("\ +", " ", text)
  text = re.sub(r'●', '-', text)
  text = re.sub(r'•', '-', text)
  text = re.sub(r'○', '-', text)
  text = re.sub(r'➤', '-', text)
  return text

def clean_pdf(text):
  text = re.sub("\t", " ", text)
  text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', text)
  text = re.sub(r'●', '-', text)
  text = re.sub(r'•', '-', text)
  text = re.sub(r'○', '-', text)
  return text


In [5]:
def save_json(json_data, name_file):
  if json_data:
    with open(f'output/{name_file}.json', 'w') as json_file:
      json.dump(obj = json_data, fp = json_file, indent=2)
    print(f"JSON data saved to output/{name_file}.json")

In [6]:
def limit_for_translator(dict_length, dict_phrase):
  limit_translation = 3000 #лимит символов для перевода
  buffer = ""
  list_offbuffer = []
  max_value = 0
  f = False
  #сохраняем в буффер объедененные строчки до 3000 символов
  for key, value in dict_length.items():
    buffer = buffer + dict_phrase[key] + "\n"
    if (value - max_value) > limit_translation: #если нашлось значение > 3000, то преобразуем список длин отрывков
      f = True
      max_value = dict_length[key-1]
      list_offbuffer.append(buffer)
      buffer = ""
      #print(max_value)

  if (buffer not in list_offbuffer) or (f == False):
    list_offbuffer.append(buffer)

  #if f == False:
    #list_offbuffer.append(buffer)
  return list_offbuffer

In [7]:
def dict_for_length_sentence(list_text):
  dict_length = {}
  dict_phrase = {}
  new_string = ""
  new_string1 = ""

  # создание словаря длин предложений и словарь с предложениями
  for key, sentens in enumerate(list_text, start=1):
    # Формируем комбинацию предложений
    new_string = sentens
    new_string1 += sentens
    # Считаем длину предложения
    len_new_string = len(new_string)
    dict_length[key] = len(new_string1)
    dict_phrase[key] = new_string
  return dict_phrase, dict_length

In [8]:
translator = Translator()

In [9]:
def translator_from_ru_to_en(text):
  list_text = text.split('\n')
  list_text = list(filter(None, list_text))
  dict_phrase, dict_length = dict_for_length_sentence(list_text)

  list_offbuffer = limit_for_translator(dict_length, dict_phrase)
  text = ""
  for phrase in list_offbuffer:
    text += translator.translate(text = phrase, dest = "en").text
  return text

In [10]:
def translator_from_en_to_ru(text, output_language):
  list_text = text.split('\n')
  list_text = list(filter(None, list_text))
  #print(list_text)
  dict_phrase, dict_length = dict_for_length_sentence(list_text)

  list_offbuffer = limit_for_translator(dict_length, dict_phrase)
  text = ""
  for phrase in list_offbuffer:
    text += translator.translate(text = phrase, dest = output_language).text
  return text

In [11]:
def processing_text(text):
  text = text.replace("\t", "\n")
  text = re.sub(r"\n\s*\n+", "\n", text)
  return text

In [12]:
def resume_for_txt(path):
  f = open(f"{path}", "r", encoding="utf-8")
  list_of_text = f.readlines()
  text = ""
  for i in list_of_text:
    text += i
  f.close()
  text = translator_from_ru_to_en(text)
  text = processing_text(text)
  return text

In [13]:
def cleen_doc(text):
  text = text.replace("|", "")
  text = re.sub("\ +", " ", text)
  return text

def clean_pdf(text):
  text = re.sub("\t", " ", text)
  text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', text)
  return text

def extract_text_from_doc(file_path):
    try:
        result = subprocess.run(['antiword', file_path], capture_output=True, text=True, encoding='utf-8')
        if result.returncode == 0:
            return result.stdout
        else:
            print(f"Error: {result.stderr}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def extract_text_from_pdf(file_path):
    try:
        text = extract_text(file_path)
        return text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def extract_text_from_docx(file_path):
    try:
        doc = Document(file_path)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + '\n'
        return text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def main_for_file(path):
  if ".docx" in path:
    text = extract_text_from_docx(path)
    text = translator_from_ru_to_en(text)
  elif ".pdf" in path:
    text = extract_text_from_pdf(path)
    text = clean_pdf(text)
    text = translator_from_ru_to_en(text)
  elif ".doc" in path:
    text = extract_text_from_doc(path)
    text = cleen_doc(text)
    text = translator_from_ru_to_en(text)
  elif ".txt" in path:
    text = resume_for_txt(path)
    text = processing_text(text)
    text = translator_from_ru_to_en(text)
  return text

# Запись примеров поведения для модели

##1

In [14]:
resume1="""
BALAMANOVA KATE




EDUCATION
+77012119111 | balamanova111@gmail.com |


International Information Technology University
Bachelor of Information Systems	Sep. 2016 – May 2022
FH Schmalkalden University of Applied Sciences
Exchange student of faculty Information Technologies	Oct. 2018 – Feb 2019


EXPERIENCE

KCELL JSC
Software Developer | Java Spring, flyway, mybatis, redis, RabbitMQ	Sep 2020 – now
    • Worked on back-end side of new corporate site of company from 0 to launch
    • Experience on project migrated from monolith to microservices
    • Started to write code review for colleagues
    • Support junior SD
Junior Software Developer |React, Java Spring, Oracle, PostgreSQL, RabbitMQ, JDBC	Jul 2019 – Aug 2020
    • Worked to optimize internal web-site which process million of users
    • E-Registry project for accountant and clients
    • Logger and validation starter
    • Work together with DevOpses to migrate projects from VM to kubernetes
Zero To One Labs
Mobile Developer Intern| React Native	Jun 2018 – Aug 2018
    • Took part in a coding boot camp where implemented various cross-platform mobile apps using technologies such as React Native, GraphQL and Firebase

AWARDS & ACHIEVEMENTS

International Student Scientiﬁc and Practical Conference "Youth, Science and Innovation - 2020", 2nd place
First President Scholarship Winner between students of our country (Scholarship for top 50 students in KZ)
Honor of Yessenov`s Foundation Scholarship for achievements of study and activities between students in the country.
3rd place: Demo Presentation at nFactorial Incubator 2018 honor application for abbiturents which help to select future career and university, got 3rd place out of approximately 100 mobile apps by the end of Bootcamp.


VOLUNTEERING & OTHER ACTIVITIES


Volunteer in “Club Dobryakov” which organizes charity events in city Almaty	2018 – 2020
Organizer and Leader of enthnic ensemble in university, every year the number of students is growing 2016 – 2020
Student Mentor of Hackathon organized by UNICEF.	2016   Sep
Volunteer on “2017 Universade Winter Sport Games” International student`s sports federation.	2017 Feb


TECHNICAL SKILLS

Programming Languages: Java, JavaScript,
Framework & Tools: Spring Boot, React, React Native, GraphQL, flyway, myBatis, Oracle, PostgreSQL, Spring Data, FeignCloud, JUnit,
Languages: English (Intermediate), Russian(Native Speaker)

"""
answer1="""
{
  "first_name": "KATE",
  "last_name": "BALAMANOVA",
  "middle_name": null,
  "birth_date": null,
  "age": null,
  "country": null,
  "city": null,
  "about": null,
  "key_skills": "Programming Languages: Java, JavaScript,;Framework & Tools: Spring Boot, React, React Native, GraphQL, flyway, myBatis, Oracle, PostgreSQL, Spring Data, FeignCloud, JUnit,;Languages: English (Intermediate), Russian(Native Speaker)",
  "salary_expectations_amount": null,
  "salary_expectations_currency": null,
  "gender": "female",
  "contactItems": [
    {
      "value": "+77012119111",
      "comment": null,
      "contact_type": "phone"
    },
    {
      "value": "balamanova111@gmail.com",
      "comment": null,
      "contact_type": "email"
    }
  ],
  "educationItems": [
    {
      "year": "2022",
      "organization": "International Information Technology University",
      "faculty": null,
      "specialty": null,
      "result": null,
      "education_type": null,
      "education_level": "Bachelor's"
    },
    {
      "year": "2019",
      "organization": "FH Schmalkalden University of Applied Sciences",
      "faculty": "Information Technologies",
      "specialty": null,
      "result": null,
      "education_type": "Exchange student",
      "education_level": null
    }
  ],
  "experienceItems": [
    {
      "starts": "Sep 2020",
      "ends": "now",
      "employer": "KCELL JSC",
      "city": null,
      "position": "Software Developer",
      "description": "• Worked on back-end side of new corporate site of company from 0 to launch; Experience on project migrated from monolith to microservices; Started to write code review for colleagues; Support junior SD."
    },
    {
      "starts": "2019",
      "ends": "2020",
      "employer": "KCELL JSC",
      "city": null,
      "position": "Junior Software Developer",
      "description": "Worked to optimize internal web-site which process million of users; E-Registry project for accountant and clients; Logger and validation starter; Work together with DevOpses to migrate projects from VM to kubernetes."
    },
    {
      "starts": "2018",
      "ends": "2018",
      "employer": "Zero To One Labs",
      "city": null,
      "position": "Mobile Developer Intern",
      "description": "Took part in a coding boot camp where implemented various cross-platform mobile apps using technologies such as React Native, GraphQL and Firebase."
    }
  ],
  "languageItems": [
    {
      "speak_language": "English",
      "language_level": "Intermediate"
    },
    {
      "speak_language": "Russian",
      "language_level": "Native Speaker"
    }
  ]
}
"""

##2

In [15]:
resume2="""
Albert SERIAKOV
Moscow, Russia • +7(968) 000-5611 • saaakov@gmail.com
Position of Interest:
Consultant, Information Technology (IT)
Of er the following qualifications:
● Establish procedures and implement policies; Consult and advise clients; Confer with clients to identify and
document requirements; Provide advice on information systems strategy, policy, management and service
delivery; Develop and implement policies and procedures throughout the software development life cycle;
● Analyze test results and instrument readings; Analyze clients' financial records; Analyze data and prepare
reports.
● Manage distributed teams, project management, coaching; maintain excellent interpersonal
communications, time management, and problem resolution skills.
Languages: ABAP, SQL
Software: Windows 7/10, Microsoft Office, Project, Visio, Power Point, ARIS, Jira, Confluence and SAP
HCM/SD/MM
Relevant Experience
Head of HCM Department, 10.2016 - Present
HCM Department, SAP ERP Division, AT Consulting, Moscow
● Pre-sale and sale, both within the framework of its direction, and in other areas of SAP practice;
● Organization and coordination of the work of SAP HCM;
● Organization of work of project teams on projects;
● Coordination of work with the Customer, negotiation and coordination of deadlines and tasks;
● Preparation and conduct of tenders, project evaluation, preparation of commercial proposals;
● IOT projects design.
Achievements:
• SAP HCM practice created;
• Developing the direction of customer experience;
• All projects were finished with positive profitability.
Leading Consultant, 12.2015 – 10.2016
HCM practice, Implementation Division, MOLGA Consulting, Moscow
● Pre-sale, project appraisal and planning, preparation of indicative and commercial proposals;
● Projects for the implementation of SAP HCM;
● Meetings with the customer on the development of design solutions. The solution of methodological issues.
Development of design solutions.
Team Leader, 02.2015 – 12.2015
HCM practice, ERP Division, Science IT University, Moscow
● Management of consultants of the PY-center;
● Negotiation with the customers;
● Development of design solutions, graphic models for describing business processes;
● Supervising the work of consultants from a subcontractor
Senior Consultant, 08.2012 – 02.2015
SAP HCM Department, HR Division, METRO Cash&Carry, Moscow
● Development and support of the SAP HCM (PA, OM, PT, PY);
● ABAP development.
Consultant, 03.2011 – 08.2012
SAP HCM Department, ERP Division, BDO Consulting, Moscow
● Development and implementation of the PW module - piecework pay (Z);
● Settings of PY changes.
Achievements:
• Developed the concept of the Z-module "Calculation of piecework wages.";
• Successfully integrated the module into the SAP HCM.
Consultant, 04.2010 – 03.2011
IT Develop Department, IT Division, Adidas, Moscow
● Development and support of the SAP HCM (PA, OM, PT, PY).
Consultant, 04.2009 – 03.2011
IT Support Department, IT Division, Kopeyka, Moscow
● Support for business processes based on SAP ERP, SD / MM modules.
Achievements:
• Successful implementation of a parallelization project for inventory accounting and sales
analysis on servers.
Education
Bachelor of Engineering – Electronic Systems Engineering, 2012
BMSTU Bauman Moscow State Technical University
Bachelor of Economy – Electronic Systems Engineering, 2016
Financial University under the Government of the Russian Federation (Diploma with Honor)
SAP CIS, SAP Certified Application Associate - SAP HCM ERP 6.0 EHP7, 2018
"""
answer2="""
{
  "resume": {
    "first_name": "Albert",
    "last_name": "SERIAKOV",
    "middle_name": null,
    "birth_date": null,
    "birth_date_year_only": null,
    "country": "Russia",
    "city": "Moscow",
    "about": null,
    "key_skills": "Establish procedures and implement policies; Consult and advise clients; Confer with clients to identify and document requirements; Provide advice on information systems strategy, policy, management and service delivery; Develop and implement policies and procedures throughout the software development life cycle;\n● Analyze test results and instrument readings; Analyze clients' financial records; Analyze data and prepare reports.\n● Manage distributed teams, project management, coaching; maintain excellent interpersonal communications, time management, and problem resolution skills.\nLanguages: ABAP, SQL\nSoftware: Windows 7/10, Microsoft Office, Project, Visio, Power Point, ARIS, Jira, Confluence and SAP HCM/SD/MM",
    "salary_expectations_amount": null,
    "salary_expectations_currency": null,
    "photo_path": null,
    "gender": null,
    "resume_name": null,
    "source_link": null,
    "contactItems": [
      {
        "value": "+7(968) 000-5611",
        "comment": null,
        "contact_type": "phone"
      },
      {
        "value": "saaakov@gmail.com",
        "comment": null,
        "contact_type": "email"
      }
    ],
    "educationItems": [
      {
        "year": 2012,
        "organization": "BMSTU Bauman Moscow State Technical University",
        "faculty": null,
        "specialty": "Electronic Systems Engineering",
        "result": null,
        "education_type": null,
        "education_level": "Bachelor"
      },
      {
        "year": 2016,
        "organization": "Financial University under the Government of the Russian Federation",
        "faculty": null,
        "specialty": "Electronic Systems Engineering",
        "result": null,
        "education_type": null,
        "education_level": "Bachelor"
      },
      {
        "year": 2018,
        "organization": "SAP CIS",
        "faculty": null,
        "specialty": "SAP HCM ERP 6.0 EHP7",
        "result": null,
        "education_type": null,
        "education_level": "SAP Certified Application Associate"
      }
    ],
    "experienceItems": [
      {
        "starts": "2016",
        "ends": "now",
        "employer": "AT Consulting",
        "city": "Moscow",
        "url": null,
        "position": "Head of HCM Department",
        "description": "Pre-sale and sale, both within the framework of its direction, and in other areas of SAP practice;Organization and coordination of the work of SAP HCM; Organization of work of project teams on projects;\n● Coordination of work with the Customer, negotiation and coordination of deadlines and tasks;\n● Preparation and conduct of tenders, project evaluation, preparation of commercial proposals;\n● IOT projects design.\nAchievements:\n• SAP HCM practice created;\n• Developing the direction of customer experience;\n• All projects were finished with positive profitability."
      },
      {
        "starts": "2015",
        "ends": "2016",
        "employer": "MOLGA Consulting",
        "city": "Moscow",
        "url": null,
        "position": "Leading Consultant",
        "description": "Pre-sale, project appraisal and planning, preparation of indicative and commercial proposals; Projects for the implementation of SAP HCM; Meetings with the customer on the development of design solutions. The solution of methodological issues.\nDevelopment of design solutions."
      },
      {
        "starts": "2015",
        "ends": "2015",
        "employer": "Science IT University",
        "city": "Moscow",
        "url": null,
        "position": "Team Leader",
        "description": "Management of consultants of the PY-center; Negotiation with the customers; Development of design solutions, graphic models for describing business processes;\n● Supervising the work of consultants from a subcontractor."
      },
      {
        "starts": "2012",
        "ends": "2015",
        "employer": "METRO Cash&Carry",
        "city": "Moscow",
        "url": null,
        "position": "Senior Consultant",
        "description": "Development and support of the SAP HCM (PA, OM, PT, PY); ABAP development."
      },
      {
        "starts": "2011",
        "ends": "2012",
        "employer": "BDO Consulting",
        "city": "Moscow",
        "url": null,
        "position": "Consultant",
        "description": "Development and implementation of the PW module - piecework pay (Z); Settings of PY changes."
      },
      {
        "starts": "2010",
        "ends": "2011",
        "employer": "Adidas",
        "city": "Moscow",
        "url": null,
        "position": "Consultant",
        "description": "Development and support of the SAP HCM (PA, OM, PT, PY)."
      },
      {
        "starts": "2009",
        "ends": "2011",
        "employer": "Kopeyka",
        "city": "Moscow",
        "url": null,
        "position": "Consultant",
        "description": "Support for business processes based on SAP ERP, SD / MM modules.Achievements:\n•Successful implementation of a parallelization project for inventory accounting and sales analysis on servers."
      }
    ],
    "languageItems": [
	  {
          "speak_language": null,
          "language_level": null
    }
    ]
  }
}
"""

##3

In [16]:
resume3 = """
Gregoriy Alexeev
UI/UX Designer
Tbilisi, Georgia / +995 000-637-978 / greg.alexeev@me.com / Portfolio
A hard-working, motivated, and experienced UI/UX Designer with a
strong online portfolio, attention to detail, and excellent management
skills. Quick learner with modern ideas and a huge knowledge base.
Having excellent communication skills I can easily fit a
cross-functional Agile team. Always ready to perform tasks quickly in
different fields like design, development, management, or analyses.
EXPERIENCE
Senior UI/UX Designer Apr 2021 - Present
Dualboot Partners, Charlotte, NC (Remote)
Designing complex user flows, researching personas, organizing design
systems for Web and Mobile (iOS, Android) applications and SaaS.
Communicating with product directors, stakeholders, and distributed
developers team. Improving design management process and mentoring
junior designers.
Senior UI/UX Designer Feb 2019 - Mar 2021
Coursly.co, Moscow
Designed an online LMS using the best practices of Interface Design in
6 months by implementing a system based on reusable components.
Increased the number of trial users by 400% with A/B testing.
Designed onboarding process and decreased a load of support center by
270%
Senior UI/UX Designer, Team Lead Sep 2016 - Feb 2019
#VA, Moscow
Managed a team of designers and developers. Worked with customers’
requirements. Designed over 35 projects based on user interviews,
prototypes and user flows. $60,000 turnover of the company in its first
year on the web design market. Built online courses with over 300
students who completed the course on UI/UX design.
Senior Web Developer Jun 2016 - May 2018
TechProMarketing, Boston, MA (Remote)
Full Stack Web Development. Landing Page and eCommerce websites design
mockups integration. SEO optimization and API integration of Stripe and
Plaid Processing. Built continuous integration (CI) for WordPress-based
websites. Managed a team of 7 developers. Writing tech specs and
documentation.
Junior UI Designer Jun 2014 - Dec 2014
AIC, Moscow
Web and mobile data-driven design (iOS & Android) of banking dashboards
and financial services. Collaborating with the team to build a complex
application design and usability testing. Designed system for the most
popular Government Services Portal with 1,6M DAU.
EDUCATION
Bachelor's degree - IT Management Sep 2015 - May 2019
Plekhanov Russian University of Economics, Moscow
Agile methodologies in IT Management and Web Technology.
SKILLS
Expert in: UI/UX Design, Interaction Design, Adobe XD, Sketch, Web
Design, Workflows, InvisionApp, Wireframes Design, Figma, Visual
Design, Zoom, Slack, SaaS, Dashboards, Product Design, HTML5, CSS3
Intermediate in: Adobe Photoshop, WordPress, JavaScript, User Research,
Customer Journey Map, Design Team Management, Prototyping & User
Stories, Problem Solving skills, Other design tools, Laravel, Vue.js,
Video Editing, Onboarding design, Moqups, Zeplin, eCommerce Design
Basic In: Python, SQL, SEO, Motion Graphics, Graphic Design
ONLINE PORTFOLIO
Check out my portfolio website with all the available projects,
case studies, and other activities. Please note, that some of the
marked projects are not for public sharing.
"""

answer3="""
{
  "first_name": "Gregoriy",
  "last_name": "Alexeev",
  "middle_name": null,
  "birth_date": null,
  "age": null,
  "country": "Georgia",
  "city": "Tbilisi",
  "about": "A hard-working, motivated, and experienced UI/UX Designer with a strong online portfolio, attention to detail, and excellent management skills. Quick learner with modern ideas and a huge knowledge base. Having excellent communication skills, I can easily fit a cross-functional Agile team. Always ready to perform tasks quickly in different fields like design, development, management, or analyses.",
  "key_skills": ["UI/UX Design","Interaction Design","Adobe XD","Sketch","Web Design","Workflows","InvisionApp","Wireframes Design","Figma","Visual Design","Zoom","Slack","SaaS","Dashboards","Product Design","HTML5","CSS3","Adobe Photoshop","WordPress","JavaScript","User Research","Customer Journey Map","Design Team Management","Prototyping & User Stories","Problem Solving skills","Other design tools","Laravel","Vue.js","Video Editing","Onboarding design","Moqups","Zeplin","eCommerce Design","Python","SQL","SEO","Motion Graphics","Graphic Design"],
  "salary_expectations_amount": null,
  "salary_expectations_currency": null,
  "gender": "male",
  "contactItems": [
    {
      "value": "+995 000-637-978",
      "comment": null,
      "contact_type": "Phone"
    },
    {
      "value": "greg.alexeev@me.com",
      "comment": null,
      "contact_type": "Email"
    }
  ],
  "educationItems": [
    {
      "years": "2019",
      "organization": "Plekhanov Russian University of Economics",
      "faculty": "IT Management",
      "specialty": null,
      "result": null,
      "education_type": null,
      "education_level": "Bachelor's"
    }
  ],
  "experienceItems": [
    {
      "starts": "2021",
      "ends": "now",
      "employer": "Dualboot Partners",
      "city": "Charlotte, NC (Remote)",
      "position": "Senior UI/UX Designer",
      "description": "Designing complex user flows, researching personas, organizing design systems for Web and Mobile (iOS, Android) applications and SaaS. Communicating with product directors, stakeholders, and distributed developers team. Improving design management process and mentoring junior designers."
    },
    {
      "starts": "2019",
      "ends": "2021",
      "employer": "Coursly.co",
      "city": "Moscow",
      "position": "Senior UI/UX Designer",
      "description": "Designed an online LMS using the best practices of Interface Design in 6 months by implementing a system based on reusable components. Increased the number of trial users by 400% with A/B testing. Designed onboarding process and decreased a load of support center by 270%."
    },
    {
      "starts": "2016",
      "ends": "2019",
      "employer": "#VA",
      "city": "Moscow",
      "position": "Senior UI/UX Designer, Team Lead",
      "description": "Managed a team of designers and developers. Worked with customers’ requirements. Designed over 35 projects based on user interviews, prototypes, and user flows. $60,000 turnover of the company in its first year on the web design market. Built online courses with over 300 students who completed the course on UI/UX design."
    },
    {
      "starts": "2016",
      "ends": "2018",
      "employer": "TechProMarketing",
      "city": "Boston, MA (Remote)",
      "position": "Senior Web Developer",
      "description": "Full Stack Web Development. Landing Page and eCommerce websites design mockups integration. SEO optimization and API integration of Stripe and Plaid Processing. Built continuous integration (CI) for WordPress-based websites. Managed a team of 7 developers. Writing tech specs and documentation."
    },
    {
      "starts": "2014",
      "ends": "2014",
      "employer": "AIC",
      "city": "Moscow",
      "position": "Junior UI Designer",
      "description": "Web and mobile data-driven design (iOS & Android) of banking dashboards and financial services. Collaborating with the team to build a complex application design and usability testing. Designed a system for the most popular Government Services Portal with 1.6M DAU."
    }
  ],
  "languageItems": [
    {
      "language": null,
      "language_level": null
    }
  ]
}

"""



##4

In [17]:
resume4 = """Sergeev Stas Antonovich
Birth date: 21.01.1993, Male
St.Petersburg, Russia
Stas@mail.ru – preferred type of conversation
@stasantony – only as a last resort
Summary


I am an experienced Flutter developer with one and half year of expertise in developing cross- platform apps. I have a strong skill set encompassing widget building, state management, and UI/UX design. My expertise extends to integrating APIs and ensuring app performance optimization. I am dedicated to thorough testing, debugging, and maintaining to ensure high-quality development.

Technical Skills

•	GetX	•	Tests	•	REST API	•	Git/version control
•	Bloc	•	Navigation	•	GraphQL	•	SOLID principles
•	Riverpod	•	Firebase	•	Hive	•	Adaptive Layout
Professional Experience

IT Solutions Management International	Remote Work
Flutter Developer	August 2023 – September 2023

•	Developed a cross-platform open source messenger in collaboration with a team of six people to define features and enhance the user experience
•	Enhanced the app by incorporating new features, which involved writing unit, widget, and end-to-end tests, adding animations of varying complexity, implementation of essential business logic and fixing bugs
•	Participated in the app design process to ensure a user-friendly and visually appealing product
•	Engaged in code review processes and employed Git version control for code

Wordly Games	Remote Work
Flutter Developer	March 2023 – July 2023

•	Developed and maintained gaming app for guessing words, for both Android and iOS platforms, ensuring compatibility and optimal performance
•	Collaborated with team of five people to define application features, functionalities, and user experience
•	Implemented deep link integration, added appropriate routing logic to ensure the app correctly handled deep link navigation
•	Participated in code reviews and Utilized Git version control for code
•	Implemented pixel-perfect screens designed on Figma
•	Optimized gaming experience by improving logic in different sections of play

Freelance	Remote Work
Flutter freelance orders	December 2022 – February 2023

•	Worked with RESTful APIs and integrated data from various sources
•	Fixed exceptions related to type of variable
•	Solved issues regarding to layout overflow
•	Improved performance of the apps
•	Handled issues with correct display of localization

Salary expectations
My salary expectations are $30,000 a month. Ready to discuss.
Education
Courses:
Flutter & Dart - The Complete Guide Flutter Foundations
English – Upper-intermediate: I can study technical documentation in origin language
Russian – Fluent	Ulyanovsk Civil Aviation Institute
December 2021
Specialization: Master of Flight operation and Air traffic Control
•	Private Pilot License
•	Commercial Pilot License
"""
answer4 = """
{
  "first_name": "Stas",
  "last_name": "Sergeev",
  "patronymic": "Antonovich",
  "birth_date": "1993-01-21",
  "age": null,
  "country_of_living": "Russia",
  "city": "St.Petersburg",
  "about": "I am an experienced Flutter developer with one and half year of expertise in developing cross-platform apps. I have a strong skill set encompassing widget building, state management, and UI/UX design. My expertise extends to integrating APIs and ensuring app performance optimization. I am dedicated to thorough testing, debugging, and maintaining to ensure high-quality development.",
  "key_skills": ["GetX","Tests","REST API","Git/version control","Bloc","Navigation","GraphQL","SOLID principles","Riverpod","Firebase","Hive","Adaptive Layout"],
  "salary_expectations_amount": 30000,
  "salary_expectations_currency": "dollars",
  "gender": "male",
  "contactItems": [
    {
      "value": "Stas@mail.ru",
      "comment": "preferred type of conversation",
      "contact_type": "Email"
    },
    {
      "value": "@stasantony",
      "comment": "only as a last resort",
      "contact_type": "Telegram"
    }

    ],
    "educationItems": [
    {
      "year": "2021",
      "organization": "Ulyanovsk Civil Aviation Institute",
      "faculty": null,
      "specialty": "Flight operation and Air traffic Control",
      "result": "Private Pilot License, Commercial Pilot License",
      "education_type": "Main",
      "education_level": "Master"
      }
    ],
    "experienceItems": [
    {
      "resume_experience_item_id": null,
      "starts": "August 2023",
      "ends": "September 2023",
      "employer": "IT Solutions Management International",
      "city": "Remote Work",
      "position": "Flutter Developer",
      "description": "Developed a cross-platform open source messenger in collaboration with a team of six people to define features and enhance the user experience;- Enhanced the app by incorporating new features, which involved writing unit, widget, and end-to-end tests, adding animations of varying complexity, implementation of essential business logic and fixing bugs;- Participated in the app design process to ensure a user-friendly and visually appealing product;- Engaged in code review processes and employed Git version control for code"
    },
    {
      "starts": "March 2023",
      "ends": "July 2023",
      "employer": "Wordly Games",
      "city": "Remote Work",
      "position": "Flutter Developer",
      "description": "Developed and maintained gaming app for guessing words, for both Android and iOS platforms, ensuring compatibility and optimal performance;- Collaborated with team of five people to define application features, functionalities, and user experience;- Implemented deep link integration, added appropriate routing logic to ensure the app correctly handled deep link navigation;- Participated in code reviews and Utilized Git version control for code;- Implemented pixel-perfect screens designed on Figma;- Optimized gaming experience by improving logic in different sections of play"
    },
    {
      "starts": "December 2022",
      "ends": "February 2023",
      "employer": null,
      "city": "Remote Work",
      "position": "Flutter freelance orders",
      "description": "Worked with RESTful APIs and integrated data from various sources;- Fixed exceptions related to type of variable;- Solved issues regarding to layout overflow;- Improved performance of the apps;- Handled issues with correct display of localization"
    }
    ],
    "languageItems": [
    {
      "language": "English",
      "language_level": "Upper-intermediate"
    },
    {
      "language": "Russian",
      "language_level": "Fluent"
    }
  ]
}
"""

##5

In [18]:
resume5 = """
Gregoriy Alexeev
UI/UX Designer
Tbilisi, Georgia / +995 000-637-978 / greg.alexeev@me.com / Portfolio
A hard-working, motivated, and experienced UI/UX Designer with a
strong online portfolio, attention to detail, and excellent management
skills. Quick learner with modern ideas and a huge knowledge base.
Having excellent communication skills I can easily fit a
cross-functional Agile team. Always ready to perform tasks quickly in
different fields like design, development, management, or analyses.
EXPERIENCE
Senior UI/UX Designer Apr 2021 - Present
Dualboot Partners, Charlotte, NC (Remote)
Designing complex user flows, researching personas, organizing design
systems for Web and Mobile (iOS, Android) applications and SaaS.
Communicating with product directors, stakeholders, and distributed
developers team. Improving design management process and mentoring
junior designers.
Senior UI/UX Designer Feb 2019 - Mar 2021
Coursly.co, Moscow
Designed an online LMS using the best practices of Interface Design in
6 months by implementing a system based on reusable components.
Increased the number of trial users by 400% with A/B testing.
Designed onboarding process and decreased a load of support center by
270%
Senior UI/UX Designer, Team Lead Sep 2016 - Feb 2019
#VA, Moscow
Managed a team of designers and developers. Worked with customers’
requirements. Designed over 35 projects based on user interviews,
prototypes and user flows. $60,000 turnover of the company in its first
year on the web design market. Built online courses with over 300
students who completed the course on UI/UX design.
Senior Web Developer Jun 2016 - May 2018
TechProMarketing, Boston, MA (Remote)
Full Stack Web Development. Landing Page and eCommerce websites design
mockups integration. SEO optimization and API integration of Stripe and
Plaid Processing. Built continuous integration (CI) for WordPress-based
websites. Managed a team of 7 developers. Writing tech specs and
documentation.
Junior UI Designer Jun 2014 - Dec 2014
AIC, Moscow
Web and mobile data-driven design (iOS & Android) of banking dashboards
and financial services. Collaborating with the team to build a complex
application design and usability testing. Designed system for the most
popular Government Services Portal with 1,6M DAU.
EDUCATION
Bachelor's degree - IT Management Sep 2015 - May 2019
Plekhanov Russian University of Economics, Moscow
Agile methodologies in IT Management and Web Technology.
SKILLS
Expert in: UI/UX Design, Interaction Design, Adobe XD, Sketch, Web
Design, Workflows, InvisionApp, Wireframes Design, Figma, Visual
Design, Zoom, Slack, SaaS, Dashboards, Product Design, HTML5, CSS3
Intermediate in: Adobe Photoshop, WordPress, JavaScript, User Research,
Customer Journey Map, Design Team Management, Prototyping & User
Stories, Problem Solving skills, Other design tools, Laravel, Vue.js,
Video Editing, Onboarding design, Moqups, Zeplin, eCommerce Design
Basic In: Python, SQL, SEO, Motion Graphics, Graphic Design
ONLINE PORTFOLIO
Check out my portfolio website with all the available projects,
case studies, and other activities. Please note, that some of the
marked projects are not for public sharing.
"""

answer5="""
{
  "first_name": "Gregoriy",
  "last_name": "Alexeev",
  "patronymic": null,
  "birth_date": null,
  "age": null,
  "country_of_living": "Georgia",
  "city": "Tbilisi",
  "about": "A hard-working, motivated, and experienced UI/UX Designer with a strong online portfolio, attention to detail, and excellent management skills. Quick learner with modern ideas and a huge knowledge base. Having excellent communication skills, I can easily fit a cross-functional Agile team. Always ready to perform tasks quickly in different fields like design, development, management, or analyses.",
  "key_skills": ["UI/UX Design","Interaction Design","Adobe XD","Sketch","Web Design","Workflows","InvisionApp","Wireframes Design","Figma","Visual Design","Zoom","Slack","SaaS","Dashboards","Product Design","HTML5","CSS3","Adobe Photoshop","WordPress","JavaScript","User Research","Customer Journey Map","Design Team Management","Prototyping & User Stories","Problem Solving skills","Other design tools","Laravel","Vue.js","Video Editing","Onboarding design","Moqups","Zeplin","eCommerce Design","Python","SQL","SEO","Motion Graphics","Graphic Design"],
  "salary_expectations_amount": null,
  "salary_expectations_currency": null,
  "gender": "male",
  "contactItems": [
    {
      "value": "+995 000-637-978",
      "comment": null,
      "contact_type": "Phone"
    },
    {
      "value": "greg.alexeev@me.com",
      "comment": null,
      "contact_type": "Email"
    }
  ],
  "educationItems": [
    {
      "years": "2019",
      "organization": "Plekhanov Russian University of Economics",
      "faculty": "IT Management",
      "specialty": null,
      "result": null,
      "education_type": null,
      "education_level": "Bachelor's"
    }
  ],
  "experienceItems": [
    {
      "starts": "2021",
      "ends": "now",
      "employer": "Dualboot Partners",
      "city": "Charlotte, NC (Remote)",
      "position": "Senior UI/UX Designer",
      "description": "Designing complex user flows, researching personas, organizing design systems for Web and Mobile (iOS, Android) applications and SaaS. Communicating with product directors, stakeholders, and distributed developers team. Improving design management process and mentoring junior designers.",
    },
    {
      "starts": "2019",
      "ends": "2021",
      "employer": "Coursly.co",
      "city": "Moscow",
      "position": "Senior UI/UX Designer",
      "description": "Designed an online LMS using the best practices of Interface Design in 6 months by implementing a system based on reusable components. Increased the number of trial users by 400% with A/B testing. Designed onboarding process and decreased a load of support center by 270%.",
    },
    {
      "starts": "2016",
      "ends": "2019",
      "employer": "#VA",
      "city": "Moscow",
      "position": "Senior UI/UX Designer, Team Lead",
      "description": "Managed a team of designers and developers. Worked with customers’ requirements. Designed over 35 projects based on user interviews, prototypes, and user flows. $60,000 turnover of the company in its first year on the web design market. Built online courses with over 300 students who completed the course on UI/UX design.",
    },
    {
      "starts": "2016",
      "ends": "2018",
      "employer": "TechProMarketing",
      "city": "Boston, MA (Remote)",
      "position": "Senior Web Developer",
      "description": "Full Stack Web Development. Landing Page and eCommerce websites design mockups integration. SEO optimization and API integration of Stripe and Plaid Processing. Built continuous integration (CI) for WordPress-based websites. Managed a team of 7 developers. Writing tech specs and documentation.",
    },
    {
      "starts": "2014",
      "ends": "2014",
      "employer": "AIC",
      "city": "Moscow",
      "position": "Junior UI Designer",
      "description": "Web and mobile data-driven design (iOS & Android) of banking dashboards and financial services. Collaborating with the team to build a complex application design and usability testing. Designed a system for the most popular Government Services Portal with 1.6M DAU.",
    }
  ],
  "languageItems": [
    {
      "language": "English",
      "language_level": null
    }
  ]
}
"""

##шаблон json для заполнения

In [19]:
json_template_glob={
  "resume_id": "",
  "first_name": "",
  "last_name": "",
  "middle_name": "",
  "birth_date": "",
  "birth_date_year_only": "",
  "country": "",
  "city": "",
  "about": "",
  "key_skills": "",
  "salary_expectations_amount": "",
  "salary_expectations_currency": "",
  "photo_path": "",
  "gender": "",
  "resume_name": "",
  "source_link": "",
  "contactItems": [
    {
      "resume_contact_item_id": "",
      "value": "",
      "comment": "",
      "contact_type": ""
    }
  ],
  "educationItems": [
    {
      "resume_education_item_id": "",
      "year": "",
      "organization": "",
      "faculty": "",
      "specialty": "",
      "result": "",
      "education_type": "",
      "education_level": ""
    }
  ],
  "experienceItems": [
    {
      "resume_experience_item_id": "",
      "starts": "",
      "ends": "",
      "employer": "",
      "city": "",
      "url": "",
      "position": "",
      "description": "",
      "order": ""
    }
  ],
  "languageItems": [
    {
      "resume_language_item_id": "",
      "language": "",
      "language_level": ""
    }
  ]
}


# Запуск модели

In [20]:
def work_model(resume):
  list_of_keys = ", ".join(list(json_template_glob.keys()))
  prompt_template=f"Perform information extraction in json from resume. Strictly follow this json key order {list_of_keys}."#: {json_tamplate}"
  prompt_template="Perform information extraction in json from resume."
  #загрузка поведенческих шаблонов
  messages = [
      # правила в виде резюме - ответ
      {"role": "user", "content": f"Resume:  {resume1}. {prompt_template}"},
      {"role": "assistant", "content": f"{answer1}"},
      #{"role": "user", "content": f"Resume:  {resume2}. {prompt_template}"},
      #{"role": "assistant", "content": f"{answer2}"},
      #{"role": "user", "content": f"Resume:  {resume3}. {prompt_template}"},
      #{"role": "assistant", "content": f"{answer3}"},
      {"role": "user", "content": f"Resume:  {resume4}. {prompt_template}"},
      {"role": "assistant", "content": f"{answer4}"},
      #{"role": "user", "content": f"Resume:  {resume5}. {prompt_template}"},
      #{"role": "assistant", "content": f"{answer5}"},

      #загрузка резюме, которе хотим обработать
      {"role": "user", "content": f"Resume: {resume}. {prompt_template}"}
    ]


  start_time = time.time()


  encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt = True, return_tensors="pt")

  model_inputs = encodeds.to(device)

  generated_ids = model.generate(model_inputs, max_new_tokens=3000, pad_token_id=tokenizer.pad_token_id)#, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)


  #фильтруем вывод
  output_model_en = ""
  time_model = ""
  pattern = re.compile(r'\[/INST\]')
  matches = pattern.finditer(decoded[0])
  indices = [match.start() for match in matches]
  output_model_en += decoded[0][indices[-1]+len('[/INST]'):]
  time_model += "--- %s seconds ---" % (time.time() - start_time)

  del generated_ids
  del model_inputs
  del encodeds
  del decoded

  return output_model_en, time_model

# Фильтрация вывода модели

Защита от глюков

In [21]:
def check_language(json_data):
  list_lang=["English", "Russian", "German", "Chinese", "Kazakh", "Japaneese", "French", "Portuguese", "Spanish"]
  try:
    json_data["languageItems"]
  except:
    return json_data
  new_arr=[]
  for sample in json_data["languageItems"]:
    if sample["language"] in list_lang:
      new_arr.append(sample["language"])

  if len(new_arr)>0:
    json_data["languageItems"]=new_arr
  else:
    json_data["languageItems"]=  [{  "resume_language_item_id": "null",  "language": "null",  "language_level": "null"}]
  return json_data

In [22]:
def clear_gluc_comma(text):
  text=text.replace("\n","")
  while "  " in text:
    text=text.replace("  "," ")
  text=text.replace(", }"," }")
  text=text.replace(",}"," }")
  return text

def clear_gluc_names(text):
  text=text.replace("experience_items","experienceItems")
  text=text.replace("contact_items","contactItems")
  text=text.replace("education_items","educationItems")
  text=text.replace("language_items","languageItems")
  return text

def clear_gluc_quotes(text):
  text=text.replace("“",'"')
  text=text.replace("”",'"')
  text = re.sub(r'(\w) "(.*?)" (\w)', r'\1 \2 \3', text)
  return text

def clear_trash(text):
    text = text.replace('\\', '')
    text = text.replace('</s>', '')
    return text

def clear_gluc(text):
  text=clear_trash(text)
  text=clear_gluc_comma(text)
  text=clear_gluc_names(text)
  text=clear_gluc_quotes(text)
  text=check_language(text)
  return text

Заполнение нескольких пунктов в json

In [23]:
#добавляем название резюме
def add_resume_name(name, dict_json_text):
  dict_json_text["resume_name"] = name
  return dict_json_text

#считаем день рождения по возрасту
def analysis_birthday(dict_json_text):
  try:
     dict_json_text["age"]
  except:
    dict_json_text["age"]=None

  dict_json_text["birth_date_year_only"] = None
  if (None != dict_json_text["age"]) and (dict_json_text["birth_date"] == None):
    dict_json_text["birth_date"] = f"{datetime.now().year - dict_json_text['age']}-01-01"
    dict_json_text["birth_date_year_only"] = True
  elif None != dict_json_text["birth_date"]:
    dict_json_text["birth_date_year_only"] = False
  return dict_json_text

преобразование файла в json

In [24]:
def convert_text_to_json(text):
    try:
        data = json.loads(text)
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None


def save_json(json_data, name_file):
  if json_data:
      with open(f'./output/{name_file}.json', 'w') as json_file:
          json.dump(json_data, json_file, indent=2)
      print(f"JSON data saved to output/{name_file}.json")


Запись вывода модели в шаблон json

In [25]:
import copy

#заполняет order по хронологии
def hronology_exp(value):
  hronology_dict={}
  for sample in value:
    hronology_dict[sample["starts"]]=sample
  sorted_dict = dict(sorted(hronology_dict.items()))
  return list(sorted_dict.values())

def convert_text_to_json(text):
    try:
        data = json.loads(text)
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None

#добавляем пункты со списками из вывода модели в изначальный шаблон
def append_removed_positions_in_list(key, value, json_template):
  if key=="educationItems":
    for sample in value:
      sample["resume_education_item_id"] = None
      sample["result"] = None

  elif key=="experienceItems":
    value=hronology_exp(value)
    for i, sample in enumerate(value):
      sample["resume_experience_item_id"] = None
      sample["url"] = None
      sample["order"] = str(i+1)

  elif key=="contactItems":
    for sample in value:
      sample["resume_contact_item_id"] = None

  elif key=="languageItems":
    for sample in value:
      sample["resume_language_item_id"] = None

  new_values=[]
  for sample in value:

    temp = json_template[key][0]
    for key_t, _ in sample.items():
      temp[key_t]=sample[key_t]
    new_values.append(copy.deepcopy(temp))

  json_template[key]=new_values

#добавляем пункты со из вывода модели в изначальный шаблон
def append_removed_positions(json_template,json_formed):
  for key, value in json_template.items():
    if type(value)==list:
      if type(value[0])==dict:
        try:
          append_removed_positions_in_list(key, json_formed[key], json_template)
        except KeyError:
          pass
    else:
      try:
        json_template[key]=json_formed[key]
      except KeyError:

        json_template[key]=None

  return json_template

#Заполняет шаблон json с помощью информации, которую нашла модель
def create_json_from_template(model_output):
  json_data=model_output

  json_template=copy.deepcopy(json_template_glob)
  append_removed_positions(json_template, json_data)
  return json_template


#MAIN

In [26]:
if not os.path.exists('input'):
  !mkdir input
if not os.path.exists('output'):
  !mkdir output

главная фунция-обработчик

In [27]:
def pipeline():

  print("Выберете файл(ы) который(ые) будет обрабатыватся(выбирать можно несколько, просто зажмите CTRL)")
  file_path, name_file = upload_file()

  for path, name in zip(file_path, name_file):
    print(f'\n_____________________Началась обработка файла {name}___________________________\n')
    #считываем резюме из файла
    resume = main_for_file(path)

    output_model, time_model = work_model(resume)
    print("Время обработки :"+time_model)

    #отчистка от глюков

    output_model=clear_gluc(output_model)
    print(output_model)

    #создает json по предоставленному примеру
    json_data = convert_text_to_json(output_model)
    json_data = add_resume_name(name, json_data)
    json_data = analysis_birthday(json_data)
    json_data = create_json_from_template(json_data)

    final_json={}
    final_json['resume']=json_data

    #сохранение в json
    save_json(final_json , name)


    print(f"\n_____________________Закончена обработка файла {name}___________________________")



#pipeline

Загрузка резюме.

Можно загрузить нейсколько штук одновременно, для этого нужно зажать cntrl и кликать по нужным резюме

In [28]:
pipeline()

Выберете файл(ы) который(ые) будет обрабатыватся(выбирать можно несколько, просто зажмите CTRL)


In [29]:
# если у вас возникает ошибка CUDA out of memory, то перезапусте среду и запустите код еще раз. Эта ошибка связана с переполнением кашем видеокары